In [21]:
#Step 1：准备数据
import pandas as pd, json
df_complex = pd.DataFrame({
    'Name': ['Alice', 'Bob', 'Charlie'],
    'Age': [25, 30, 35],
    'Salary': [50000.0, 100000.5, 150000.75],
    'IsMarried': [True, False, True]
})

df_complex_json = df_complex.to_json(orient='split')
df_complex_json

'{"columns":["Name","Age","Salary","IsMarried"],"index":[0,1,2],"data":[["Alice",25,50000.0,true],["Bob",30,100000.5,false],["Charlie",35,150000.75,true]]}'

In [25]:
#Step 2：设定需求: 让大模型计算这个数据集中所有人的年龄总和。
# Step 3：编写计算年龄总和的函数
from io import StringIO
def calculate_total_age_from_split_json(input_json):
    """
    从给定的JSON格式字符串（按'split'方向排列）中解析出DataFrame，计算所有人的年龄总和，并以JSON格式返回结果。

    参数:
    input_json (str): 包含个体数据的JSON格式字符串。

    返回:
    str: 所有人的年龄总和，以JSON格式返回。
    """

    # 将JSON字符串转换为DataFrame
    df = pd.read_json(StringIO(input_json), orient='split')

    # 计算所有人的年龄总和
    total_age = df['Age'].sum()

    # 将结果转换为字符串形式，然后使用json.dumps()转换为JSON格式
    return json.dumps({"total_age": str(total_age)})


In [26]:
#step4: 功能测试： 使用函数计算年龄总和，并以JSON格式输出
result = calculate_total_age_from_split_json(df_complex_json)
print("The JSON output is:", result)


The JSON output is: {"total_age": "90"}


In [27]:
# Step 5：定义函数库
# 在Chat模型实际执行Function calling功能时，是从一个函数库中筛选合适的函数进行调用，因此需要准备一个函数库。最简单的情况下，函数库可以只包含一个函数，如下：
# 函数库对象必须是一个字典，一个键值对代表一个函数，其中Key是代表函数名称的字符串，而value表示对应的函数。所以上述过程可以简单的理解为：所谓的外部函数库，就是用一个大的字典来存储某应用场景中的所需要的所有函数定义
function_repository = {
            "calculate_total_age_from_split_json": calculate_total_age_from_split_json,}


In [28]:
# step 6: 将外部函数传递给大模型的第一步，就是要对每一个编写好的功能函数定义其对应的JSON Schema描述。
# **针对刚才定义计算年龄总和的函数calculate_total_age_from_split_json，编写其JSON Schema描述，如下：
calculate_total_age_from_split_json = {"name": "calculate_total_age_from_split_json",
                                       "description": "计算年龄总和的函数，从给定的JSON格式字符串（按'split'方向排列）中解析出DataFrame，计算所有人的年龄总和，并以JSON格式返回结果。",
                                       "parameters": {"type": "object",
                                       "properties": {"input_json": {"type": "string",
                                                                     "description": "执行计算年龄总和的数据集"},},
                                        "required": ["input_json"],
                                    },
                     }
# 必须编写calculate_total_age_from_split_json[“parameters”]的原因，是因为在Chat模型在进行对话时，传入和传出数据信息都是以类似JSON数据格式进行传输的

In [29]:
# step 7: 当定义了外部函数仓库、功能函数及功能函数对应的JSON Schema对象描述后，准备工作基本就做完了。接下来就可以进行与大模型的数据交互，具体来说就是在对话参数的基础上补充两个额外参数：
from openai import OpenAI
client = OpenAI()

functions = [calculate_total_age_from_split_json]
def chat_completions(parameter_message):
    response = client.chat.completions.create(model="gpt-3.5-turbo-0125",
        messages = parameter_message,
        functions=functions,# 增加这两行
        function_call="auto",  )
    return response.choices[0].message

In [30]:
# Step 8：构建messages
messages=[
    {"role": "system", "content": "你是一位优秀的数据分析师, 现在有这样一个数据集input_json：%s，数据集以JSON形式呈现" % df_complex_json},
    {"role": "user", "content": "请在数据集input_json上执行计算所有人年龄总和函数"}
]

#Step 9：传入模型，让其自动选择函数并完成计算
response = chat_completions(messages)
response

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"input_json":"{\\"columns\\":[\\"Name\\",\\"Age\\",\\"Salary\\",\\"IsMarried\\"],\\"index\\":[0,1,2],\\"data\\":[[\\"Alice\\",25,50000.0,true],[\\"Bob\\",30,100000.5,false],[\\"Charlie\\",35,150000.75,true]]}"}', name='calculate_total_age_from_split_json'), tool_calls=None)

In [36]:
# Step 10：保存交互过程中的关键信息
# 保存交互过程中的函数名称
function_name = response.function_call.name
print(function_name)
# 加载交互过程中的参数
function_args = json.loads(response.function_call.arguments)
function_args

calculate_total_age_from_split_json


{'input_json': '{"columns":["Name","Age","Salary","IsMarried"],"index":[0,1,2],"data":[["Alice",25,50000.0,true],["Bob",30,100000.5,false],["Charlie",35,150000.75,true]]}'}

In [38]:
# Step 11：保存函数对象
# 保存具体的函数对象
local_fuction_call = function_repository[function_name]
local_fuction_call

<function __main__.calculate_total_age_from_split_json(input_json)>

In [39]:
# Step 12：完成模型计算
final_response = local_fuction_call(**function_args)
final_response

'{"total_age": "90"}'

In [40]:
# Step 13：追加messages构建流程
# 追加第一次模型返回结果消息
messages.append(response)
# 追加function计算结果，注意：function message必须要输入关键词name
messages.append({"role": "function", "name": function_name, "content": final_response,})
print(messages)



[{'role': 'system', 'content': '你是一位优秀的数据分析师, 现在有这样一个数据集input_json：{"columns":["Name","Age","Salary","IsMarried"],"index":[0,1,2],"data":[["Alice",25,50000.0,true],["Bob",30,100000.5,false],["Charlie",35,150000.75,true]]}，数据集以JSON形式呈现'}, {'role': 'user', 'content': '请在数据集input_json上执行计算所有人年龄总和函数'}, ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"input_json":"{\\"columns\\":[\\"Name\\",\\"Age\\",\\"Salary\\",\\"IsMarried\\"],\\"index\\":[0,1,2],\\"data\\":[[\\"Alice\\",25,50000.0,true],[\\"Bob\\",30,100000.5,false],[\\"Charlie\\",35,150000.75,true]]}"}', name='calculate_total_age_from_split_json'), tool_calls=None), {'role': 'function', 'name': 'calculate_total_age_from_split_json', 'content': '{"total_age": "90"}'}]


In [41]:
# Step 14: 再次向Chat Completion 模型提问
last_response = client.chat.completions.create(model="gpt-3.5-turbo-0125",
        messages = messages)
last_response

ChatCompletion(id='chatcmpl-95Jpbhbv7BYPQIinG0gfC1pNdIern', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='数据集input_json中所有人的年龄总和为90岁。', role='assistant', function_call=None, tool_calls=None))], created=1711054899, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_fa89f7a861', usage=CompletionUsage(completion_tokens=17, prompt_tokens=225, total_tokens=242))

In [44]:
last_response.choices[0].message.content

'数据集input_json中所有人的年龄总和为90岁。'